# 🎥 Halalificator - Google Colab Edition

**A complete video processing pipeline that:**
- 🔒 **Blurs females** using AI-powered gender detection (CLIP or Caffe models)
- 🎵 **Removes music** while preserving vocals using Demucs

---

## Quick Start
1. **Run all setup cells** (Sections 1-5) - just click each play button
2. **Upload your video** in Section 6
3. **Configure options** in Section 7 using the interactive form
4. **Run processing** in Section 8
5. **Download result** in Section 9

---

## 1️⃣ Environment Check & GPU Setup

In [ ]:
#@title 🔧 Check Environment { display-mode: "form" }
#@markdown **Run this cell first!** It checks if you're on Colab and if GPU is available.

import sys
import os

# Check if running on Colab
IN_COLAB = 'google.colab' in sys.modules

print("=" * 50)
print("🔍 ENVIRONMENT CHECK")
print("=" * 50)
print(f"\n✓ Running in Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Check GPU
    import subprocess
    try:
        gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], text=True)
        print(f"✓ GPU Available: Yes")
        print(f"  GPU Info: {gpu_info.strip()}")
    except:
        print("⚠️ GPU Not Available!")
        print("   Go to Runtime → Change runtime type → Select 'T4 GPU'")
else:
    print("⚠️ Not running in Colab. Some features may not work.")

print(f"\n✓ Python Version: {sys.version.split()[0]}")
print("\n" + "=" * 50)
print("✅ Environment check complete!")
print("=" * 50)

## 2️⃣ Install Dependencies

This installs all required Python packages. Takes about 2-3 minutes on first run.

In [ ]:
#@title 📦 Install All Dependencies { display-mode: "form" }
#@markdown This will install: OpenCV, YOLO, CLIP, Demucs, FFmpeg, and more.
#@markdown 
#@markdown **⏱️ Takes ~2-3 minutes on first run**

print("📦 Installing dependencies...\n")

# Install core packages
!pip install -q --upgrade pip
!pip install -q opencv-python-headless>=4.8.0
!pip install -q ultralytics>=8.0.0
!pip install -q open-clip-torch>=2.20.0
!pip install -q demucs>=4.0.0
!pip install -q static-ffmpeg>=2.5
!pip install -q torch torchvision torchaudio
!pip install -q ipywidgets

# Enable widgets in Colab
if IN_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

print("\n" + "=" * 50)
print("✅ All dependencies installed!")
print("=" * 50)

## 3️⃣ Import Libraries

In [ ]:
#@title 📚 Import Libraries { display-mode: "form" }

import cv2
import numpy as np
from ultralytics import YOLO
import urllib.request
import shutil
import subprocess
from pathlib import Path
from static_ffmpeg import add_paths
import torch
import open_clip
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Verify GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✓ PyTorch using device: {device}")
if device == "cuda":
    print(f"  GPU: {torch.cuda.get_device_name(0)}")

# Create directories
os.makedirs('inputs', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

print("\n✅ All libraries imported successfully!")

## 4️⃣ Setup Processing Functions

In [ ]:
#@title 🔧 Define Helper Functions { display-mode: "form" }
#@markdown Model download URLs and utility functions

# Model URLs
FACE_MODEL_URL = "https://huggingface.co/arnabdhar/YOLOv8-Face-Detection/resolve/main/model.pt"
FACE_MODEL_NAME = "yolov8n-face.pt"
GENDER_PROTO_URL = "https://huggingface.co/AjaySharma/genderDetection/resolve/main/gender_deploy.prototxt"
GENDER_MODEL_URL = "https://huggingface.co/AjaySharma/genderDetection/resolve/main/gender_net.caffemodel"
GENDER_PROTO = "gender_deploy.prototxt"
GENDER_MODEL = "gender_net.caffemodel"

def download_file(url, filename):
    """Download a file if it doesn't exist."""
    if not os.path.exists(filename):
        print(f"⬇️ Downloading {filename}...")
        urllib.request.urlretrieve(url, filename)
        print(f"   ✅ Downloaded")
    else:
        print(f"✓ Found {filename}")

def setup_models(mode='clip'):
    """Download required model files."""
    download_file(FACE_MODEL_URL, FACE_MODEL_NAME)
    if mode == 'caffe':
        download_file(GENDER_PROTO_URL, GENDER_PROTO)
        download_file(GENDER_MODEL_URL, GENDER_MODEL)

def blur_region(image, box):
    """Applies Gaussian blur to bounding box region."""
    x1, y1, x2, y2 = map(int, box)
    h, w = image.shape[:2]
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)
    roi = image[y1:y2, x1:x2]
    if roi.size == 0: return image
    ksize = int(max(roi.shape[:2]) // 5) | 1
    if ksize <= 1: return image
    blurred_roi = cv2.GaussianBlur(roi, (ksize, ksize), 0)
    image[y1:y2, x1:x2] = blurred_roi
    return image

def draw_debug(image, face_box, person_box, gender, conf, is_tracked_female=False, track_id=None):
    """Draws debug bounding boxes."""
    if face_box:
        fx1, fy1, fx2, fy2 = face_box
        color = (255, 0, 0) if gender == 'Male' else (255, 0, 255)
        cv2.rectangle(image, (fx1, fy1), (fx2, fy2), color, 2)
        cv2.putText(image, f"{gender} {conf:.2f}", (fx1, fy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    if person_box is not None:
        px1, py1, px2, py2 = person_box
        p_color = (0, 0, 255) if is_tracked_female else (0, 255, 0)
        cv2.rectangle(image, (px1, py1), (px2, py2), p_color, 2)
        id_label = f"ID:{track_id}" if track_id else "ID:?"
        label = f"{id_label} FEMALE" if is_tracked_female else id_label
        cv2.putText(image, label, (px1, py1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, p_color, 2)

class GlobalTrackManager:
    """Manages gender votes across frames for consistent classification."""
    def __init__(self, sensitivity=0.15):
        self.track_gender_votes = {}
        self.final_decisions = {}
        self.sensitivity = sensitivity

    def add_vote(self, track_id, gender):
        if track_id is None: return
        if track_id not in self.track_gender_votes:
            self.track_gender_votes[track_id] = {'Male': 0, 'Female': 0}
        self.track_gender_votes[track_id][gender] += 1

    def finalize(self):
        print("\n📊 Finalizing Gender Tracks:")
        for tid, votes in self.track_gender_votes.items():
            total = votes['Male'] + votes['Female']
            if total == 0: continue
            female_ratio = votes['Female'] / total
            self.final_decisions[tid] = 'Female' if female_ratio >= self.sensitivity else 'Male'
            print(f"   Track {tid}: {votes} → {self.final_decisions[tid]}")

print("✅ Helper functions defined!")

In [ ]:
#@title 🎬 Define Video Processing Function { display-mode: "form" }

def process_video(input_path, output_path, conf_threshold=0.25, start_seconds=0,
                  duration_seconds=None, debug=False, mode='clip', progress_callback=None, **kwargs):
    """Process video: detect and blur females using two-pass approach."""
    setup_models(mode=mode)
    
    print(f"\n🔧 Loading models (mode={mode})...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    face_model = YOLO(FACE_MODEL_NAME)
    person_model = YOLO("yolov8n.pt")
    
    gender_net = None
    clip_model, clip_preprocess, clip_tokenizer = None, None, None
    
    if mode == 'caffe':
        gender_net = cv2.dnn.readNet(GENDER_PROTO, GENDER_MODEL)
    elif mode == 'clip':
        clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
            'ViT-B-32', pretrained='laion2b_s34b_b79k', device=device)
        clip_tokenizer = open_clip.get_tokenizer('ViT-B-32')
    
    gender_list = ['Male', 'Female']
    global_tracker = GlobalTrackManager(sensitivity=kwargs.get('sensitivity', 0.15))
    
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"❌ Error opening video {input_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / fps
    
    if start_seconds < 0:
        start_seconds = max(0, video_duration + start_seconds)
    start_frame = int(start_seconds * fps)
    max_frames = int(duration_seconds * fps) if duration_seconds else (total_frames - start_frame)
    if start_frame + max_frames > total_frames:
        max_frames = total_frames - start_frame
    
    print(f"📽️ Processing: {start_seconds:.1f}s to {start_seconds + (max_frames/fps):.1f}s ({max_frames} frames)")
    
    # --- PASS 1: Analysis ---
    print(f"\n🔍 PASS 1: Analyzing...")
    frame_count = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret: break
        
        results = person_model.track(frame, verbose=False, classes=[0], persist=True, conf=conf_threshold)
        
        if results and results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)
            
            if mode == 'clip':
                text_prompts = clip_tokenizer(["a photo of a man", "a photo of a woman"]).to(device)
                for i, (px1, py1, px2, py2) in enumerate(boxes):
                    person_img = frame[max(0,py1):min(height,py2), max(0,px1):min(width,px2)]
                    if person_img.size > 0:
                        person_pil = Image.fromarray(cv2.cvtColor(person_img, cv2.COLOR_BGR2RGB))
                        image_input = clip_preprocess(person_pil).unsqueeze(0).to(device)
                        with torch.no_grad():
                            image_features = clip_model.encode_image(image_input)
                            text_features = clip_model.encode_text(text_prompts)
                            probs = (100.0 * image_features @ text_features.T).softmax(dim=-1).cpu().numpy()[0]
                            gender = 'Male' if probs[0] > probs[1] else 'Female'
                            global_tracker.add_vote(ids[i], gender)
            else:
                face_results = face_model(frame, verbose=False, conf=conf_threshold)
                for result in face_results:
                    for box in result.boxes:
                        bx1, by1, bx2, by2 = map(int, box.xyxy[0])
                        face_cx, face_cy = (bx1+bx2)/2, (by1+by2)/2
                        face_img = frame[max(0,by1):min(height,by2), max(0,bx1):min(width,bx2)]
                        if face_img.size > 0:
                            blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227),
                                (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
                            gender_net.setInput(blob)
                            preds = gender_net.forward()
                            gender = gender_list[preds[0].argmax()]
                            for i, (px1, py1, px2, py2) in enumerate(boxes):
                                h_margin = (py2 - py1) * 0.2
                                if px1 <= face_cx <= px2 and (py1 - h_margin) <= face_cy <= py2:
                                    global_tracker.add_vote(ids[i], gender)
                                    break
        
        frame_count += 1
        if frame_count % 30 == 0:
            pct = int(frame_count / max_frames * 50)
            print(f"   Pass 1: {frame_count}/{max_frames} ({pct}%)", end='\r')
            if progress_callback:
                progress_callback(pct)

    global_tracker.finalize()

    # --- PASS 2: Rendering ---
    print(f"\n🎬 PASS 2: Rendering...")
    person_model = YOLO("yolov8n.pt")
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    frame_count = 0
    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret: break
        
        results = person_model.track(frame, verbose=False, classes=[0], persist=True, conf=conf_threshold)
        
        if results and results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)
            
            for box, track_id in zip(boxes, ids):
                gender_decision = global_tracker.final_decisions.get(track_id, 'Male')
                if gender_decision == 'Female':
                    if debug:
                        draw_debug(frame, None, box, None, 1.0, True, track_id=track_id)
                    else:
                        frame = blur_region(frame, box)
                elif debug:
                    draw_debug(frame, None, box, None, 0.0, False, track_id=track_id)

        out.write(frame)
        frame_count += 1
        if frame_count % 30 == 0:
            pct = 50 + int(frame_count / max_frames * 50)
            print(f"   Pass 2: {frame_count}/{max_frames} ({pct}%)", end='\r')
            if progress_callback:
                progress_callback(pct)
            
    cap.release()
    out.release()
    print("\n✅ Video processing complete!")

print("✅ Video processing function defined!")

In [ ]:
#@title 🎵 Define Audio Processing Function { display-mode: "form" }

def find_demucs():
    """Locate the demucs executable."""
    candidate = Path(sys.executable).parent / "demucs"
    if candidate.exists() and os.access(candidate, os.X_OK):
        return str(candidate)
    path_demucs = shutil.which("demucs")
    return path_demucs if path_demucs else "demucs"

def remove_music(input_path, output_path=None, model="htdemucs"):
    """Separates vocals from audio using Demucs."""
    input_path = Path(input_path).resolve()
    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {input_path}")

    print(f"\n🎵 Processing audio: {input_path.name}")
    
    demucs_cmd = find_demucs()
    env = os.environ.copy()
    try:
        add_paths()
        env["PATH"] = os.environ["PATH"]
    except ImportError:
        pass

    cmd = [demucs_cmd, "--two-stems=vocals", "-n", model, str(input_path)]
    print(f"   Running Demucs (this may take several minutes)...")
    subprocess.run(cmd, check=True, env=env)
    
    filename_stem = input_path.stem
    output_root = Path("separated") / model
    candidate_dir = output_root / filename_stem
    vocals_file = candidate_dir / "vocals.wav"
    
    if not vocals_file.exists():
        if output_root.exists():
            for child in output_root.iterdir():
                if child.is_dir():
                    vocals_file = child / "vocals.wav"
                    if vocals_file.exists():
                        candidate_dir = child
                        break
    
    if not vocals_file.exists():
        raise RuntimeError(f"Could not locate separated output")

    if output_path:
        output_path = Path(output_path).resolve()
        output_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.move(str(vocals_file), str(output_path))
        target_file = output_path
    else:
        target_file = input_path.with_name(f"{input_path.stem}_vocals.wav")
        shutil.move(str(vocals_file), str(target_file))

    try:
        shutil.rmtree(candidate_dir)
        output_root.rmdir()
        output_root.parent.rmdir()
    except OSError:
        pass

    print(f"   ✅ Vocals saved to: {target_file}")
    return str(target_file)

def combine_video_audio(video_path, audio_path, output_path):
    """Combines video and audio using ffmpeg."""
    print(f"\n🔗 Merging video and audio...")
    add_paths()
    cmd = ["ffmpeg", "-y", "-i", video_path, "-i", audio_path,
           "-c:v", "copy", "-c:a", "aac", "-map", "0:v:0", "-map", "1:a:0",
           "-shortest", output_path]
    try:
        subprocess.run(cmd, check=True, capture_output=True)
        print("   ✅ Merge complete!")
    except subprocess.CalledProcessError as e:
        print(f"   ⚠️ Merge failed, saving video only")
        shutil.move(video_path, output_path)

print("✅ Audio processing functions defined!")

## 5️⃣ Define Main Pipeline

In [ ]:
#@title 🚀 Define Halalify Pipeline { display-mode: "form" }

def halalify(input_path, output_path, audio_path=None, progress_callback=None, **kwargs):
    """Full pipeline: Blur females AND remove music."""
    temp_blurred = "temp_blurred_video.mp4"
    temp_vocals = "temp_vocals.wav"
    
    print("\n" + "=" * 60)
    print("🎥 HALALIFICATOR PROCESSING PIPELINE")
    print("=" * 60)
    
    # 1. Process Video
    print("\n📹 STEP 1: Processing Video")
    print("-" * 40)
    process_video(input_path, temp_blurred,
                  conf_threshold=kwargs.get('conf', 0.25),
                  start_seconds=kwargs.get('start', 0),
                  duration_seconds=kwargs.get('duration'),
                  debug=kwargs.get('debug', False),
                  sensitivity=kwargs.get('sensitivity', 0.15),
                  mode=kwargs.get('mode', 'clip'),
                  progress_callback=progress_callback)
    
    # 2. Process Audio
    print("\n🎵 STEP 2: Processing Audio")
    print("-" * 40)
    try:
        audio_source = audio_path if audio_path else input_path
        remove_music(audio_source, output_path=temp_vocals)
        combine_video_audio(temp_blurred, temp_vocals, output_path)
    except Exception as e:
        print(f"\n⚠️ Audio processing failed: {e}")
        shutil.move(temp_blurred, output_path)
    
    # Cleanup
    for f in [temp_blurred, temp_vocals]:
        if os.path.exists(f): os.remove(f)
    
    print("\n" + "=" * 60)
    print(f"✅ COMPLETE: {output_path}")
    print(f"   File size: {os.path.getsize(output_path) / (1024*1024):.2f} MB")
    print("=" * 60)

print("✅ Halalify pipeline defined!")
print("\n" + "=" * 60)
print("🎉 SETUP COMPLETE! Proceed to upload your video.")
print("=" * 60)

---

## 6️⃣ Upload Your Video

Choose one of the methods below to get your video into Colab.

In [ ]:
#@title 📤 Option A: Upload from Computer { display-mode: "form" }
#@markdown Click the **Choose Files** button below to upload your video.

INPUT_VIDEO = None

if IN_COLAB:
    from google.colab import files
    print("📤 Click the button below to upload your video file:")
    print("   (Supported: MP4, MKV, AVI, MOV, WebM)\n")
    uploaded = files.upload()
    
    if uploaded:
        INPUT_VIDEO = list(uploaded.keys())[0]
        file_size = os.path.getsize(INPUT_VIDEO) / (1024*1024)
        print(f"\n✅ Uploaded: {INPUT_VIDEO} ({file_size:.2f} MB)")
    else:
        print("❌ No file uploaded")
else:
    print("⚠️ Not in Colab. Use Option B or C instead.")

In [ ]:
#@title 🌐 Option B: Download from URL { display-mode: "form" }
#@markdown Enter a direct download URL for your video:

video_url = ""  #@param {type:"string"}
output_filename = "downloaded_video.mp4"  #@param {type:"string"}

if video_url:
    print(f"⬇️ Downloading from: {video_url[:50]}...")
    !wget -q -O "{output_filename}" "{video_url}"
    
    if os.path.exists(output_filename):
        file_size = os.path.getsize(output_filename) / (1024*1024)
        INPUT_VIDEO = output_filename
        print(f"\n✅ Downloaded: {INPUT_VIDEO} ({file_size:.2f} MB)")
    else:
        print("❌ Download failed")
else:
    print("ℹ️ Enter a URL above and run this cell to download")

In [ ]:
#@title 📺 Option C: Download from YouTube { display-mode: "form" }
#@markdown Enter a YouTube URL to download:

youtube_url = ""  #@param {type:"string"}
output_name = "youtube_video"  #@param {type:"string"}

if youtube_url:
    print("📦 Installing yt-dlp...")
    !pip install -q yt-dlp
    
    print(f"\n⬇️ Downloading from YouTube...")
    !yt-dlp -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best" \
        -o "{output_name}.%(ext)s" "{youtube_url}" --no-playlist
    
    # Find the downloaded file
    for ext in ['mp4', 'mkv', 'webm']:
        candidate = f"{output_name}.{ext}"
        if os.path.exists(candidate):
            INPUT_VIDEO = candidate
            file_size = os.path.getsize(INPUT_VIDEO) / (1024*1024)
            print(f"\n✅ Downloaded: {INPUT_VIDEO} ({file_size:.2f} MB)")
            break
    else:
        print("❌ Download failed or file format not found")
else:
    print("ℹ️ Enter a YouTube URL above and run this cell to download")

In [ ]:
#@title ✅ Verify Selected Video { display-mode: "form" }
#@markdown Run this to confirm which video will be processed:

# You can also manually set the video path here:
manual_path = ""  #@param {type:"string"}

if manual_path:
    INPUT_VIDEO = manual_path

print("=" * 50)
print("📹 SELECTED VIDEO")
print("=" * 50)

if INPUT_VIDEO and os.path.exists(INPUT_VIDEO):
    file_size = os.path.getsize(INPUT_VIDEO) / (1024*1024)
    print(f"\n✅ Video: {INPUT_VIDEO}")
    print(f"   Size: {file_size:.2f} MB")
    
    # Get video info
    cap = cv2.VideoCapture(INPUT_VIDEO)
    if cap.isOpened():
        fps = cap.get(cv2.CAP_PROP_FPS)
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frames / fps if fps > 0 else 0
        print(f"   Duration: {duration:.1f} seconds ({frames} frames)")
        print(f"   Resolution: {width}x{height} @ {fps:.1f} FPS")
        cap.release()
    print("\n✅ Ready to proceed to configuration!")
else:
    print("\n❌ No video selected or file not found!")
    print("   Please use one of the upload options above.")

---

## 7️⃣ Configure Processing Options

Adjust these settings before running the processor.

In [ ]:
#@title ⚙️ Processing Configuration { display-mode: "form" }

#@markdown ### Output Settings
output_filename = "halalified_output.mp4"  #@param {type:"string"}

#@markdown ---
#@markdown ### Gender Detection Mode
detection_mode = "clip"  #@param ["clip", "caffe"]
#@markdown - **clip**: Uses CLIP model on full body (recommended, more robust)
#@markdown - **caffe**: Uses face-based detection (requires clear face visibility)

#@markdown ---
#@markdown ### Detection Parameters
confidence_threshold = 0.25  #@param {type:"slider", min:0.1, max:0.9, step:0.05}
#@markdown Confidence threshold for person detection (lower = more detections)

sensitivity = 0.15  #@param {type:"slider", min:0.05, max:0.5, step:0.05}
#@markdown Gender sensitivity (lower = more aggressive blurring)

#@markdown ---
#@markdown ### Processing Range (for testing)
start_time = 0  #@param {type:"number"}
#@markdown Start from this second (use negative for end, e.g., -5 for last 5 seconds)

duration = 0  #@param {type:"number"}
#@markdown Duration in seconds (0 = full video)

#@markdown ---
#@markdown ### Debug Mode
debug_mode = False  #@param {type:"boolean"}
#@markdown If enabled, draws bounding boxes instead of blurring (for testing)

# Store configuration
OUTPUT_VIDEO = f"outputs/{output_filename}"
MODE = detection_mode
CONF_THRESHOLD = confidence_threshold
SENSITIVITY = sensitivity
START_TIME = start_time
DURATION = duration if duration > 0 else None
DEBUG_MODE = debug_mode

print("=" * 50)
print("⚙️ CONFIGURATION SUMMARY")
print("=" * 50)
print(f"\n📹 Input:  {INPUT_VIDEO}")
print(f"📁 Output: {OUTPUT_VIDEO}")
print(f"\n🔧 Mode: {MODE}")
print(f"   Confidence: {CONF_THRESHOLD}")
print(f"   Sensitivity: {SENSITIVITY}")
print(f"\n⏱️ Range: {START_TIME}s to {'end' if DURATION is None else f'{START_TIME + DURATION}s'}")
print(f"🐛 Debug: {DEBUG_MODE}")
print("\n" + "=" * 50)
print("✅ Configuration saved! Ready to process.")
print("=" * 50)

---

## 8️⃣ Run Processing

This will process your video. Time depends on video length and GPU availability.

In [ ]:
#@title 🚀 Run Halalificator! { display-mode: "form" }
#@markdown Click the play button to start processing.
#@markdown 
#@markdown **Estimated time**: ~2-5 minutes per minute of video (with GPU)

import time

if not INPUT_VIDEO or not os.path.exists(INPUT_VIDEO):
    print("❌ ERROR: No input video selected!")
    print("   Please upload a video in Section 6 first.")
else:
    start_time_process = time.time()
    
    halalify(
        INPUT_VIDEO,
        OUTPUT_VIDEO,
        mode=MODE,
        conf=CONF_THRESHOLD,
        sensitivity=SENSITIVITY,
        start=START_TIME,
        duration=DURATION,
        debug=DEBUG_MODE
    )
    
    elapsed = time.time() - start_time_process
    print(f"\n⏱️ Total processing time: {elapsed/60:.1f} minutes")

---

## 9️⃣ Preview & Download Result

In [ ]:
#@title 🎬 Preview Output Video { display-mode: "form" }
#@markdown Plays the processed video inline (for videos under 50MB)

if os.path.exists(OUTPUT_VIDEO):
    file_size = os.path.getsize(OUTPUT_VIDEO) / (1024*1024)
    print(f"📁 Output: {OUTPUT_VIDEO}")
    print(f"   Size: {file_size:.2f} MB\n")
    
    if IN_COLAB and file_size < 50:
        from base64 import b64encode
        print("🎬 Video Preview:")
        mp4 = open(OUTPUT_VIDEO, 'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        display(HTML(f'''
        <video width="640" controls>
            <source src="{data_url}" type="video/mp4">
        </video>
        '''))
    elif file_size >= 50:
        print("⚠️ Video too large for inline preview. Please download to view.")
    else:
        print(f"ℹ️ Open {OUTPUT_VIDEO} with your video player to preview.")
else:
    print("❌ Output video not found. Please run processing first (Section 8).")

In [ ]:
#@title 📥 Download Output Video { display-mode: "form" }
#@markdown Downloads the processed video to your computer.

if os.path.exists(OUTPUT_VIDEO):
    if IN_COLAB:
        from google.colab import files
        print("📥 Starting download...")
        files.download(OUTPUT_VIDEO)
        print("\n✅ Download initiated! Check your browser downloads.")
    else:
        print(f"📁 Output saved to: {os.path.abspath(OUTPUT_VIDEO)}")
else:
    print("❌ Output video not found. Please run processing first (Section 8).")

---

## 🔧 Advanced Options

Use these if you want to run only part of the pipeline.

In [ ]:
#@title 📹 Video Only (No Audio Processing) { display-mode: "form" }
#@markdown Blurs females but skips music removal (faster)

video_only_output = "outputs/blurred_only.mp4"  #@param {type:"string"}
run_video_only = False  #@param {type:"boolean"}

if run_video_only and INPUT_VIDEO:
    process_video(
        INPUT_VIDEO,
        video_only_output,
        conf_threshold=CONF_THRESHOLD,
        start_seconds=START_TIME,
        duration_seconds=DURATION,
        debug=DEBUG_MODE,
        sensitivity=SENSITIVITY,
        mode=MODE
    )
    print(f"\n✅ Video saved to: {video_only_output}")
else:
    print("ℹ️ Enable 'run_video_only' checkbox and run this cell to process video only.")

In [ ]:
#@title 🎵 Audio Only (Music Removal) { display-mode: "form" }
#@markdown Removes music from audio/video without any video processing

audio_only_output = "outputs/vocals_only.wav"  #@param {type:"string"}
run_audio_only = False  #@param {type:"boolean"}

if run_audio_only and INPUT_VIDEO:
    remove_music(INPUT_VIDEO, output_path=audio_only_output)
    print(f"\n✅ Vocals saved to: {audio_only_output}")
else:
    print("ℹ️ Enable 'run_audio_only' checkbox and run this cell to extract vocals only.")

---

## ❓ Troubleshooting

| Issue | Solution |
|-------|----------|
| **No GPU available** | Go to `Runtime` → `Change runtime type` → Select `T4 GPU` |
| **Out of memory** | Use shorter duration, reduce video resolution, or restart runtime |
| **Slow processing** | Ensure GPU is enabled; CLIP mode is slower but more accurate |
| **Audio processing fails** | May happen if no audio track; video will still be processed |
| **Download doesn't start** | Try right-clicking the download link or use Files panel (left sidebar) |